In [150]:
%matplotlib inline
from __future__ import print_function
import argparse, gzip, cPickle, sys, time, itertools

import autograd.numpy as np
import autograd.numpy.random as npr
import autograd.scipy.stats.norm as norm
import autograd.scipy.stats.dirichlet as dirichlet
from autograd.scipy.misc import logsumexp
from autograd.util import flatten_func, flatten
from autograd import grad, primitive
from autograd.numpy.numpy_grads import unbroadcast

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix
from sklearn import mixture

from  autograd.scipy.special import gammaln, digamma, gamma
from scipy import linalg
from scipy import stats, integrate

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.animation as animation
import pandas as pd
import six


color_names = ["maroon",               
               "gold",
               "royal blue"]
colors = sns.xkcd_palette(color_names)
sns.set_style("white")
color_iter = itertools.cycle(colors)

def plot_results(ax, X, Y, means, covariances, index, title):    
    for i, (mean, covar, color) in enumerate(zip(
             means, covariances, color_iter)):
        v, w = linalg.eigh(np.diag(np.full([2], covar)))
        v = 2. * np.sqrt(2.) * np.sqrt(v)
        u = w[0] / linalg.norm(w[0])      

        if not np.any(Y == i):
            continue
        ax.scatter(X[Y == i, 0], X[Y == i, 1], 2., color=color)

        angle = np.arctan(u[1] / u[0])
        angle = 180. * angle / np.pi  # convert to degrees
        ell = mpl.patches.Ellipse(mean, v[0], v[1], 180. + angle, color=color)
        #ell.set_clip_box(splot.bbox)
        ell.set_alpha(0.5)
        ax.add_artist(ell)
        
        
class Adam(object):
    def __init__(self, dparam, b1=0.9, b2=0.999, eps=10**-8,
                         decay_rate = 0.9, decay_steps = 100):                    
        self.b1 = b1;
        self.b2 = b2;
        self.eps = eps        
        self.m = np.zeros(dparam)
        self.v = np.zeros(dparam)
        self.i = 0
        self.decay_rate = decay_rate
        self.decay_steps = decay_steps
    
    def update(self, gradients, params, learning_rate = 0.1):        
        self.i = self.i+1
        step_size = learning_rate * self.decay_rate**(self.i/self.decay_steps)        
        self.m = (1 - self.b1) * gradients + self.b1 * self.m
        self.v = (1 - self.b2) * (gradients**2) + self.b2 * self.v
        mhat = self.m / (1 - self.b1**(self.i))
        vhat = self.v / (1 - self.b2**(self.i))                        
        params = params + step_size*mhat/(np.sqrt(vhat) + self.eps)        
        return np.split(params,2)

@primitive
def softplus(x):
    return np.log(1. + np.exp(x))

softplus.defvjp(lambda g, ans, vs, gvs, x: unbroadcast(vs, gvs, g * 1./(1. + np.exp(-x))))

def jacobian_softplus(x):
    return 1./(1. + np.exp(-x))

@primitive
def gamma_logpdf(x, alpha = 1., beta = 1.):
    return  (alpha*np.log(beta) + (alpha - 1)*np.log(x) - x*beta - gammaln(alpha))

gamma_logpdf.defvjp(lambda g, ans, vs, gvs, x, alpha=1.0, beta=1.0: unbroadcast(vs, gvs, g * ((alpha-1)/x - beta)))
gamma_logpdf.defvjp(lambda g, ans, vs, gvs, x, alpha=1.0, beta=1.0: unbroadcast(vs, gvs,  g * (np.log(beta) + np.log(x) - digamma(alpha))), argnum=1)
gamma_logpdf.defvjp(lambda g, ans, vs, gvs, x, alpha=1.0, beta=1.0: unbroadcast(vs, gvs,  g * (alpha/beta - x)), argnum=2)

cumprod = primitive(np.cumprod)

def grad_np_cumprod(g, ans, vs, gvs, x, axis=None):
    fx = np.cumprod(x, axis=None)
    return np.cumsum((fx * g)[::-1])[::-1].reshape(x.shape) / x

cumprod.defvjp(grad_np_cumprod)

def invlogit(x, eps=sys.float_info.epsilon):
    return (1 - 2 * eps) / (1 + np.exp(-x)) + eps

def logit(p):
    return np.log(p / (1 - p))

def stick_breaking(y_):
    y = y_.T
    Km1 = y.shape[0]
    k = np.arange(Km1)[(slice(None), ) + (None, ) * (y.ndim - 1)]
    eq_share = logit(1. / (Km1 + 1 - k).astype(str(y_.dtype)))
    z = invlogit(y + eq_share, 1e-3)
    yl = np.concatenate([z, np.ones(y[:1].shape)])
    yu = np.concatenate([np.ones(y[:1].shape), 1 - z])
    S = cumprod(yu, 0)
    x = S * yl
    return x.T

def jacobian_stick_breaking(y_):
    y = y_.T
    Km1 = y.shape[0]
    k = np.arange(Km1)[(slice(None), ) + (None, ) * (y.ndim - 1)]
    eq_share = logit(1. / (Km1 + 1 - k).astype(str(y_.dtype)))
    yl = y + eq_share
    yu = np.concatenate([np.ones(y[:1].shape), 1 - invlogit(yl, 1e-3)])
    S = cumprod(yu, 0)
    return -np.sum(np.log(S[:-1]) - np.log1p(np.exp(yl)) - np.log1p(np.exp(-yl)), 0).T

In [151]:
class GMM(object):
    def __init__(self, data, clusters, scale):
        self.data = data     
        self.clusters = clusters
        self.scale = scale
        self.N = data.shape[0]
        self.D = data.shape[1]        
    
    def p_log_prob(self, idx, z):
        x = self.data[idx]        
        mu, tau, pi = z['mu'], softplus(z['tau']), stick_breaking(z['pi'])
        matrix = []  
        log_prior = 0.
        log_prior += np.sum(gamma_logpdf(tau, 1e-5, 1e-5) + np.log(jacobian_softplus(z['tau'])))        
        log_prior += np.sum(norm.logpdf(mu, 0, 1.))
        log_prior += dirichlet.logpdf(pi, 1e3 * np.ones(self.clusters)) + np.log(jacobian_stick_breaking(z['pi']))
        for k in range(self.clusters):
            matrix.append(np.log(pi[k]) + np.sum(norm.logpdf(x, mu[(k * self.D):((k + 1) * self.D)],
                                np.full([self.D], 1./np.sqrt(tau[k]))), 1))
        matrix  = np.vstack(matrix)
        vector = logsumexp(matrix, axis=0)
        log_lik = np.sum(vector)        
        return self.scale * log_lik + log_prior        
    
    def predict(self, z):
        x = self.data
        mu, tau, pi = z['mu'], softplus(z['tau']), stick_breaking(z['pi'])        
        matrix = []                
        for k in range(self.clusters):
            matrix.append(np.log(pi[k]) + np.sum(norm.logpdf(x, mu[(k * self.D):((k + 1) * self.D)],
                                 np.full([self.D], 1./np.sqrt(tau[k]))), 1))
        matrix  = np.vstack(matrix)                
        return np.argmax(matrix, 0)    
    
 
    def q_log_prob(self, means, log_sigmas, z):
        q_w = np.sum(norm.logpdf(z, means, np.exp(log_sigmas)))        
        return q_w
      
    def q_log_prob_sep(self, means, log_sigmas, z):
        q_w = norm.logpdf(z, means, np.exp(log_sigmas))        
        return q_w

    def phi_log_prob_sep(self, eps):
        phi_w = norm.logpdf(eps, 0., 1.)        
        return phi_w

    def sample_q(self, means, log_sigmas, d):        
        eps = npr.randn(d)        
        q_s = np.exp(log_sigmas) * eps + means
        return (q_s, eps)
        
    def grad_params(self, dp_log_prob, eps, log_sigmas):                
        grad_means = dp_log_prob
        grad_log_sigmas = dp_log_prob*eps*np.exp(log_sigmas) + 1                
        return np.concatenate([grad_means, grad_log_sigmas])
        
    def calc_eps(self, means, log_sigma, z):        
        eps  = (z - means)/np.exp(log_sigma)
        return eps            

In [152]:
class Inference(object):      
    def __init__(self, model, params):
        self.model = model
        self.params = params

    def run(self, epochs, batch_size, samples, learning_rate, algorithm = 'SGD', optimizer = 'adam'):
        epochs = epochs
        batches = self.model.N/batch_size
        batch_size = batch_size
        samples = samples
        learning_rate = learning_rate        
        
        means, unflatten = flatten(self.params['means'])
        log_sigmas, unflatten = flatten(self.params['log_sigmas'])        
        D =len(means)

        self.F = np.zeros(epochs * batches)
        self.time = np.zeros(epochs * batches)
        adam = Adam(2*D)
        f = 0
        
        grad_p_log_prob = grad(model.p_log_prob, argnum = 1)
        grad_q_log_prob = grad(model.q_log_prob, argnum = 1)
        
        if algorithm == 'SGD':
            for e in range(epochs):
                for b in range(batches): 
                    start = time.clock()
                    losses = 0.
                    d_elbo = 0.
                    idx = np.random.choice(np.arange(self.model.N), batch_size, replace=False)                    
                    d_elbo = 0.

                    for s in range(samples):
                        eps = npr.randn(D)        
                        z = np.exp(log_sigmas) * eps + means                                            
                        p_log_prob = model.p_log_prob(idx, unflatten(z))                        
                        dp_log_prob, _ = flatten(grad_p_log_prob(idx, unflatten(z)))
                        g =  model.grad_params(dp_log_prob, eps, log_sigmas)                        
                        d_elbo += g
                        q_log_prob = model.q_log_prob(means, log_sigmas, z)                                         
                        losses +=  (p_log_prob - q_log_prob)                    
                    loss = losses/samples
                    d_elbo /= samples   
                    means_old, log_sigmas_old = means, log_sigmas
                    means, log_sigmas = adam.update(d_elbo, np.concatenate([means, log_sigmas]), learning_rate)
                    if np.sum(np.isnan(means)) > 0 or np.sum(np.isnan(log_sigmas)) > 0:
                        means, log_sigmas = means_old, log_sigmas_old
                        learning_rate = learning_rate * .1                        
                    self.F[f] =  -loss                

                    stop = time.clock()
                    self.time[f] = stop - start
                    f+=1
                if e % 1 == 0:
                    pstate = 'Epoch = ' + "{0:0=3d}".format(e) + ': Loss = {0:.3f}'.format(self.F[f-1])
                    print (pstate, end = '\r')
                    sys.stdout.flush()   
                                                                                                              
        if algorithm == 'iSGD':
            n = 1.  
            z_old = [0.] * samples
            dp_log_prob_old = [0.] * samples
            phi_log_prob_old = [0.] * samples            

            for e in range(epochs):
                for b in range(batches): 
                    start = time.clock()
                    losses = 0.
                    d_elbo = 0.
                    idx = np.random.choice(np.arange(self.model.N), batch_size, replace=False)                                        
                    
                    if n > .9:
                        for s in range(samples):
                            eps = npr.randn(D)        
                            z = np.exp(log_sigmas) * eps + means
                            p_log_prob = model.p_log_prob(idx, unflatten(z))
                            q_log_prob = model.q_log_prob_sep(means, log_sigmas, z)                                         
                            dp_log_prob, _ = flatten(grad_p_log_prob(idx, unflatten(z)))
                            g =  model.grad_params(dp_log_prob, eps, log_sigmas)                        
                            d_elbo += g
                            losses +=  (p_log_prob - np.sum(q_log_prob))
                                        
                            z_old[s] = z
                            dp_log_prob_old[s] = dp_log_prob
                            phi_log_prob_old[s] = model.phi_log_prob_sep(eps)                                                                     
                        loss = losses/samples
                        d_elbo /= samples                                            
                    else:                         
                        for s in range(samples):                            
                            eps = (z_old[s] - means)/np.exp(log_sigmas)                            
                            phi_log_prob = model.phi_log_prob_sep(eps)                                         
                            w = np.exp(phi_log_prob - phi_log_prob_old[s])                            
                            g =  model.grad_params(w * dp_log_prob_old[s], eps, log_sigmas)                        
                            d_elbo += g                            
                        d_elbo /= samples                                             
                    n = npr.uniform()
                    means_old, log_sigmas_old = means, log_sigmas
                    means, log_sigmas = adam.update(d_elbo, np.concatenate([means, log_sigmas]), learning_rate)
                    if np.sum(np.isnan(means)) > 0 or np.sum(np.isnan(log_sigmas)) > 0:
                        means, log_sigmas = means_old, log_sigmas_old
                        learning_rate = learning_rate * .9   
                        n = 1.
                    self.F[f] =  -loss                
                    stop = time.clock()
                    self.time[f] = stop - start
                    f+=1                                        
                if e % 1 == 0:
                    pstate = 'Epoch = ' + "{0:0=3d}".format(e) + ': Loss = {0:.3f}'.format(self.F[f-1])
                    print (pstate, end = '\r')
                    sys.stdout.flush()  
        self.params = {'means': unflatten(means), 'log_sigmas': unflatten(log_sigmas)}

In [153]:
N = n_samples = 10000
epochs = 10
samples = 1
learning_rate = 0.1
K = clusters = 25
seed = 222
batch_size = 2000
D = 2


npr.seed(seed)
c =  5 * npr.randn(K * D).reshape([K,D])
m = np.tile(c, (N/K,1))
X =  npr.randn(N * D).reshape([N,D]) + m

In [162]:
batch_size = 500
epochs = 50
samples = 1
learning_rate = .1
seed = 111
data = X
    
N = data.shape[0]
D = data.shape[1]
model = GMM(data, K, N/batch_size)

        
#f, (ax1, ax2, ax3) = plt.subplots(3,1,figsize=(30,30))


npr.seed(seed)    
params = {}
params['means'] = {'mu': np.ones(K*D), 'tau': 1e-2 * npr.randn(K), 'pi':  npr.randn(K-1)}
params['log_sigmas'] = {'mu': 1e-2 * npr.randn(K*D), 'tau': 1e-2 * npr.randn(K), 'pi': 1e-2 * npr.randn(K-1)}
inference = Inference(model, params)
inference.run(40, batch_size, samples, learning_rate, 'SGD')
plt.plot(np.cumsum(inference.time), -inference.F, color = colors[1],  label = "SGD")
# p = model.predict(inference.params['means'])
# means_ = inference.params['means']['mu'].reshape([clusters, D])
# covariances_ = 1/np.sqrt(np.exp(inference.params['means']['tau']))
# pi_ = stick_breaking(inference.params['means']['pi'])
# plot_results(ax2, X, p, means_, covariances_, 0, 'Bayesian GMM')



npr.seed(seed)    
params = {}
params['means'] = {'mu': np.ones(K*D), 'tau':  1e-2 * npr.randn(K), 'pi':  npr.randn(K-1)}
params['log_sigmas'] = {'mu': 1e-2 * npr.randn(K*D), 'tau': 1e-2 * npr.randn(K), 'pi': 1e-2 * npr.randn(K-1)}
inference = Inference(model, params)
inference.run(310, batch_size, samples, learning_rate, 'iSGD')
plt.plot(np.cumsum(inference.time), -inference.F, color = colors[0], label = "I-SGD")
# p = model.predict(inference.params['means'])
# means_ = inference.params['means']['mu'].reshape([clusters, D])
# covariances_ = 1/np.sqrt(np.exp(inference.params['means']['tau']))
# pi_ = stick_breaking(inference.params['means']['pi'])
# plot_results(ax3, X, p, means_, covariances_, 0, 'Bayesian GMM')


fig = plt.gcf()
fig.set_size_inches(4,3)
plt.ylabel('ELBO', fontsize = 15)
plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
plt.xlabel('Time[s]', fontsize = 15)
plt.xticks(fontsize = 10)
plt.yticks(fontsize = 10)
plt.legend(loc = 4, fontsize = 15)
plt.savefig('/home/sakaya/MUPI/papers/uai17importance/gmm.png', dpi=300, bbox_inches= 'tight')

KeyboardInterrupt: 